解析简单的XML数据

从一个简单的XML文档中提取数据

可以使用 xml.etree.ElementTree 模块从简单的 XML 文档中提取数据。为了演·示

In [13]:
from xml.etree.ElementTree import parse, parse
from collections import Counter

doc = parse('xml_3.xml')

for name in doc.iterfind('channel/item'):
    print(name.findtext('title'))
    print('---')


Steve Holden: Python for Data Analysis
---
Vasudev Ram: The Python Data model (for v2 and v3)
---
Python Diary: Been playing around with Object Databases
            
---


In [14]:
for name in doc.iterfind('food'): # food/ -> None ; food/home -> None
    print(name.findtext('name'))
    print('---')

xml.etree.ElementTree.parse() 函数解析整个 XML 文档并将其转换成一个文
档对象。然后，你就能使用 find() 、iterfind() 和 findtext() 等方法来搜索特定的
XML 元素了。这些函数的参数就是某个指定的标签名，例如 channel/item 或 title 。

每次指定某个标签时，你需要遍历整个文档结构。每次搜索操作会从一个起始元
素开始进行。同样，每次操作所指定的标签名也是起始元素的相对路径。例如，执行
doc.iterfind('channel/item') 来搜索所有在 channel 元素下面的 item 元素。doc
代表文档的最顶层 (也就是第一级的 rss 元素)。然后接下来的调用 item.findtext()
会从已找到的 item 元素位置开始搜索。

ElementTree 模块中的每个元素有一些重要的属性和方法，在解析的时候非常有
用。tag 属性包含了标签的名字，text 属性包含了内部的文本，而 get() 方法能获取
属性值。例如：

In [15]:
doc

In [20]:
e = doc.find('channel/title')
e, e.tag, e.text, e.get('some_attribute')

(<Element 'title' at 0x000001963F253CC8>, 'title', 'Planet Python', None)

In [ ]:
# from lxml.etree import parse

有一点要强调的是 xml.etree.ElementTree 并不是 XML 解析的唯一方法。对于
更高级的应用程序，你需要考虑使用 lxml 。它使用了和 ElementTree 同样的编程接
口，因此上面的例子同样也适用于 lxml。你只需要将刚开始的 import 语句换成 from
lxml.etree import parse 就行了。lxml 完全遵循 XML 标准，并且速度也非常快，同
时还支持验证，XSLT，和 XPath 等特性

## 增量式解析大型 XML 文件

你想使用尽可能少的内存从一个超大的 XML 文档中提取数据

任何时候只要你遇到增量式的数据处理时，第一时间就应该想到迭代器和生成器。
下面是一个很简单的函数，只使用很少的内存就能增量式的处理一个大型 XML 文件

In [24]:
 from xml.etree.ElementTree import iterparse
    
def parse_and_remove(filename, path):
    path_parts = path.split('/')
    doc = iterparse(filename, ('start', 'end'))
    next(doc)
    
    tag_stack = []
    elem_stack = []
    
    for event, elem in doc:
        if event == 'start':
            tag_stack.append(elem.tag)
            elem_stack.append(elem)
        elif event == 'end':
            if tag_stack == path_parts:
                yield elem
                elem_stack[-2].remove(elem)
            try:
                tag_stack.pop()
                elem_stack.pop()
            except IndexError:
                pass

为了测试这个函数，你需要先有一个大型的 XML 文件。通常你可以在政府网站或
公共数据网站上找到这样的文件。例如，你可以下载 XML 格式的芝加哥城市道路坑洼
数据库。在写这本书的时候，下载文件已经包含超过 100,000 行数据，编码格式类似于
下面这样

.....

假设你想写一个脚本来按照坑洼报告数量排列邮编号码。你可以像这样做

这个脚本唯一的问题是它会先将整个 XML 文件加载到内存中然后解析。在我的机
器上，为了运行这个程序需要用到 450MB 左右的内存空间。如果使用如下代码，程序
只需要修改一点点：

In [32]:
data = parse_and_remove('xml_3.xml', 'channel/item')
for i in data:
    print(i.findtext('title'))

Steve Holden: Python for Data Analysis
Vasudev Ram: The Python Data model (for v2 and v3)
Python Diary: Been playing around with Object Databases
            
